## 전처리

### 필요 환경 설치 및 확인

In [ ]:
#Python ver.확인
import sys
print(sys.version)

In [ ]:
import platform 
print(platform.architecture())

In [ ]:
!pip install JPype1-1.4.0-cp39-cp39-win_amd64.whl #JPype1 설치(python, platform ver.에 맞춰서)

In [ ]:
import os

In [ ]:
print('JAVA_HOME' in os.environ) #True 나와야 함!

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np

In [ ]:
sheet = pd.read_csv(r'우리_크롤링_전처리전.csv', encoding = 'utf-8-sig') #크롤링 완료한 파일 불러오기
sheet

### Null 값 확인

In [ ]:
#혹시 Null 값을 가진 데이터가 있는지 확인 
print(sheet.isnull().sum())

In [ ]:
#만약 위에서 Null값이 나오면 공백으로 수정
sheet['review'] = sheet['review'].fillna('')

print(sheet.isnull().values.any()) # 제거되었는지 확인: False 나와야 함!

### 중복데이터 확인/제거

In [ ]:
#중복 데이터 개수 확인
sheet['review'].duplicated().sum()

In [ ]:
#먄약 중복 데이터가 있다면 제거
sheet = sheet.drop_duplicates(['review'])
sheet.reset_index(inplace = True)
sheet = sheet.iloc[:,[4]] #review column만 가져오기
sheet

### 블로그 광고글 삭제 +  직장 & 육아 관련글 삭제

- **광고글 특징**
    - 1> 특정 단어를 포함한 게시글: "제공받아", "제공받았", "지원받아", "지원받았", "해당업체", "해시태그" 등 포함 시 광고글로 볼 수 있음
    - 2> 게시글 길이가 3000자 이상이면 광고글 상위 노출 조건에 해당하여 광고글로 볼 수 있음
    - 3> 블로그 관련 특정 단어 추가 삭제
##### 
- **직장 & 육아 관련 키워드로 원문 제거**
    - 직장인, 직장, 회사, 육아, 남편, 아내, 부인, 와이프... 

In [ ]:
#광고글 특징을 Stopwords로 지정하여 해당 글은 삭제할 수 있도록 함
#Stopwords에 해당 문구가 들어가면 삭제 대상이 되도록 문구를 추가해줌


Stopwords = ['소정의','무료제공','아래 링크','플러스친구','제휴마케팅','수수료','파트너스','경품','당첨','작성되었습니다'
             '스마트 스토어','쿠팡 파트너스','제품을 제공받고 솔직하게','링크에서 주문가능해요','무상제공',
             '쿠팡파트너스','이벤트','대가없이','대가 없이','댓가없이','댓가없이','플러스 친구', '원고료를', '제공받아',
             '직장인', '직장', '회사', '육아', '남편', '아내', '부인', '와이프'] # 광고 관련 + 직장 & 육아 관련 키워드 추가

In [ ]:
#Stopwords가 들어간 글은 삭제

clean_sheet = sheet[~sheet['review'].str.contains('|'.join(Stopwords))]
clean_sheet

In [ ]:
#게시글 길이가 3천 자 이상면 광고글 상위 노출 조건 -> 제거하기
ix = clean_sheet['review'].str.len() > 3000
clean_sheet_2 = clean_sheet.loc[~ix]
clean_sheet_2

In [ ]:
#블로그 관련 특정 '단어' 삭제 (더 추가하셔도 됩니다!)

def clean_str(text):
    
    remove_w = ['복사', '본문 기타 기능', '블로그씨', '블챌', '주간일기', '챌린지', '이웃추가', '공유하기', '신고하기', '본문',
                '플레이어 접기', '동영상 좋아요', '님 사진 및 동영상 개 팔로워 명 팔로잉 명 게시물 개', 'Loading...', '접기', '펴기',
                '5분 이상', '소요될 수 있으며', '영상의 길이에 따라', '소요 시간은 달라집니다', 'Previous image', 'Next image',
                '더보기 /OpenStreetMap지도 데이터x© NAVER Corp. /OpenStreetMap지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가',
                'NAVER Corp.', '50m©', '144p', '270p', '360p', '480p', '720p', '1080p', '게시판', '화질 선택 옵션자동', '동영상', 
                '사진 및 동영상 팔로워 만명 팔로잉 명 게시물 개', '님의 사진 및 동영상 보기', '게시판', '광고정보', 'HD', 
                '라이센스', 'URL 복사', '지도로 보기', '전체지도', '지도닫기', '레이어 닫기', '도움말 플레이어', '주소복사HTML 소스',
                'Band Twitter Facebook Blog Cafe', 'SNS 공유소스 코드', '네이버 지도', '도움말라이센스Video Statistic Version 4.29.4 (HLS)Build Date',
                'Flash', '미지원으로', 'Object','코드', '퍼가기', '기능은 더 이상 지원되지않습니다.', '음소거 \|', '재생 수', 'm NAVER Corp 데이터x NAVER Corp',
                'OpenStreetMap지도', '범례부동산거리읍 면 동시 군 구시 도국가', 'www youtube com', '링크', 'smartstore', 'https',
                '더보기초 후 SKIP초 후 재생SKIP광고 후 계속됩니다.', '544 x 306740 x 416936 x 5261280 x 720', 'URL', '둘러보기', 'm 데이터x',
                'topleft rightbottom마우스를 드래그 하세요360°', '화질', '자동', 'm NAVER Corp', 'OpenStreetMap지도', 'HLS', 'Build Date',
                '데이터x NAVER Corp OpenStreetMap지도', '컨트롤러', '범례부동산거리읍 면 동시 군 구시 도국가', 'naver', 
                'Video도움말', 'MULTI TRACK' , '멀티트랙의', '트랙별', '영상은', '우측 관련영상에서 볼 수 있습니다', '선거광고 안내', 'topleft', 'rightbottom',
                'Loading', 'player', 'likesound', 'mutesound', 'volume', 'sound', 'Player', 'Size', '마우스를 드래그 하세요', 
                'Current Resolution', 'PMax Abr Resolution', 'PProtocol', 'hlsCurrent' 'Bandwidth', 'Average Bandwidth', 'Buffer Health', 'Frame Drop Rate', 'Video Statistic Version',
                '인코딩 중입니다', '분 이상 소요될 수 있으며 영상의 길이에 따라 소요 시간은 달라집니다', 'x x', 'hlsCurrent Bandwidth HLS Build Date'
                'Video', '음소거', 'p p p p p p 초 후', 'SKIP초 후', '재생SKIP광고 후', '계속됩니다', '주소 HTML', '소스', 'x x x x',
                '도움말', 'Video Statistic Version HLS Build Date NAVER Corp', '인코딩|', 'm NAVER Corp 데이터x', 'NAVER Corp', '데이터x', 'm 데이터x'
                'Video도움말', '더보기', '전체화면', '서이추환영', '서이추', '서로이웃', '서로이웃환영', '이웃환영', '광고정보',
                'Instagram', '사진 및 팔로워 명', '팔로잉 명', '게시물 개', '님의 Instagram', '사진 및 보기', 'www instagram com']
    
    for i in remove_w:
            if i in text:
                print(i)
                text = text.replace(i, ' ')
                
    return text

In [ ]:
clean_sheet_2['review'] = clean_sheet_2['review'].apply(lambda x : clean_str(x))

### 정규식-1

In [ ]:
import re

def extract_word(text) :
    hangul = re.compile('[^가-힣a-zA-Z]')
    result = hangul.sub(' ', text)
    return result

In [ ]:
clean_sheet_2['review'] = clean_sheet_2['review'].apply(lambda x : extract_word(x))

### 정규식-2

In [ ]:
import re

a = re.compile("\S*@\S+")
b = re.compile("#\S+")
c = re.compile("https?://\S+")
d = re.compile("www\.\S+")

def cleanse(text):
    result = a.sub("", text)
    result = b.sub("", result)
    result = c.sub("", result)
    result = d.sub("", result)
    result = result.replace("\u200b", "")
    result = result.replace("\n", "")
    return result

In [ ]:
clean_sheet_2['review'] = clean_sheet_2['review'].apply(lambda x : cleanse(x))

In [ ]:
clean_sheet_2

In [ ]:
clean_sheet_2.reset_index(drop=True, inplace = True)

### 추가 전처리

In [ ]:
corpus = []
for i in range(len(clean_sheet_2['review'])):
    review = clean_sheet_2['review'][i].lower() #lower case
    corpus.append(review)
clean_sheet_2['review'] = corpus

In [ ]:
clean_sheet_2

### 띄어쓰기 처리

In [ ]:
def re_spacing(text):
    result = text.strip()
    while "  " in result:
        result = result.replace("  ", " ")
    return result

In [ ]:
clean_sheet_2['review'] = clean_sheet_2['review'].apply(lambda x : re_spacing(x))

In [ ]:
clean_sheet_2
clean_sheet_2.reset_index(drop=True)

### 전처리 완료 후 Dataframe으로 변환

In [ ]:
final_sheet = clean_sheet_2.review
final_sheet = final_sheet.reset_index(drop=True)

In [ ]:
final_sheet = pd.DataFrame(final_sheet)

In [ ]:
final_sheet

### CSV 파일로 저장

In [ ]:
final_sheet.to_csv('우리_크롤링_전처리후.csv', encoding = 'utf-8-sig') #파일명 수정(전처리) 필요